In [1]:
import pandas as pd
import polars as pl
import toml
from pathlib import Path
from sqlalchemy import create_engine

config = toml.load(Path.cwd() / '../../../../configuration/input_configuration.toml')
summary_config = toml.load(Path.cwd() / '../../../../configuration/summary_configuration.toml')

pd.set_option('display.float_format', '{:,.1f}'.format)

In [2]:
# Relative path between notebooks and goruped output directories
output_path = Path(summary_config['sc_run_path']) / summary_config["output_folder"]
survey_path = Path(summary_config['sc_run_path']) / summary_config["survey_folder"]

In [3]:
transit_jobs_access = pd.read_csv(output_path / 'access/transit_jobs_access.csv', 
                                  usecols=['geography', 'value', 'geography_group'])
walk_bike_jobs_access = pd.read_csv(output_path / 'access/walk_bike_jobs_access.csv', 
                                  usecols=['geography_value', 'jobs_1_mile_walk', 'jobs_3_mile_bike', 'geography_group']).\
                                  rename(columns={'geography_value': 'geography'})

parcel_emp = pl.read_csv(output_path / 'landuse/parcels_urbansim.txt', 
                         separator=' ', 
                         columns=['parcelid','emptot_p']).to_pandas()


# jobs access in equity geographies
equity_geogs = summary_config['equity_geogs_comb']

## Jobs Accessible within 45 Minutes of Transit

In [4]:
def process_access_data(jobs_access):
    df_access = jobs_access.copy()
    # rename region
    df_access.loc[jobs_access['geography_group'] == 'region', 'geography'] = 'Region'
    # rename rgc
    df_access.loc[jobs_access['geography_group'] == 'rgc_binary', 'geography'] = ['Not in RGC', 'In RGC']

    df_access_equity_geogs = df_access.loc[
        (df_access['geography_group'].isin(equity_geogs)) & (df_access['geography']=='1')].copy()
    df = df_access.loc[
        (df_access['geography_group'].isin(equity_geogs)) & (df_access['geography']=='0')].copy()

    df_access_equity_geogs['geography'] = df_access_equity_geogs['geography_group']
    df['geography'] = "NOT in " + df['geography_group']

    df_access_equity_geogs = pd.concat([df_access_equity_geogs, df], ignore_index=True)
    df_access_equity_geogs['geography_group'] = 'Equity Geography'

    return df_access, df_access_equity_geogs


df_access_t, df_access_equity_t = process_access_data(transit_jobs_access)
df_access_bp, df_access_equity_bp = process_access_data(walk_bike_jobs_access)
tot_jobs = parcel_emp['emptot_p'].sum()

In [5]:
def job_access_geog(access_table,geog):
    df = access_table.loc[access_table['geography_group'].isin([geog, 'region'])].\
        rename(columns={'value': 'Jobs within 45-minute Transit Commute'}).\
        drop(columns=['geography_group']).\
        set_index('geography')

    df['% Total Jobs'] = df['Jobs within 45-minute Transit Commute'].apply(lambda x: f'{x / tot_jobs * 100:,.1f}' + '%')

    return df


In [6]:
job_access_geog(df_access_t,'CountyName')

,Jobs within 45-minute Transit Commute,% Total Jobs
geography,,
King,"231,303.4",10.7%
Kitsap,"7,048.8",0.3%
Outside Region,0.0,0.0%
Pierce,"19,363.2",0.9%
Snohomish,"26,509.9",1.2%
Region,"136,789.6",6.3%


In [7]:
df_rgc = job_access_geog(df_access_t,'rgc_binary')
df = job_access_geog(df_access_t,'GrowthCenterName')

pd.concat([df_rgc, df.loc[~df.index.isin(['Region','Not in RGC'])]], axis=0)

,Jobs within 45-minute Transit Commute,% Total Jobs
geography,,
Region,"136,789.6",6.3%
Not in RGC,"101,267.7",4.7%
In RGC,"471,330.9",21.9%
Auburn,"66,547.7",3.1%
Bellevue,"483,169.3",22.4%
Bothell Canyon Park,"38,962.7",1.8%
Bremerton,"39,380.7",1.8%
Burien,"121,981.5",5.7%
Everett,"62,050.8",2.9%


In [8]:
job_access_geog(df_access_t,'rg_proposed')

,Jobs within 45-minute Transit Commute,% Total Jobs
geography,,
Region,"136,789.6",6.3%
CitiesTowns,"10,553.6",0.5%
Core,"58,582.4",2.7%
HCT,"36,827.6",1.7%
Metro,"337,668.7",15.7%
UU,"10,059.1",0.5%


In [9]:
job_access_geog(df_access_equity_t,'Equity Geography')

,Jobs within 45-minute Transit Commute,% Total Jobs
geography,,


## Average Jobs Accessible within 1 Mile Walk and 3 Mile Bike
Note that this is not using the bike network, but is instead using the all-streets network.

Average accessible jobs are weighted averages based on parcel household population.

In [10]:
def bp_job_access_geog(access_table,geog):
    df = access_table.loc[access_table['geography_group'].isin(['region', geog])].\
        rename(columns={'jobs_1_mile_walk': 'Jobs within 1-mile Walk',
                        'jobs_3_mile_bike': 'Jobs within 3-mile Bike'}).\
        drop(columns=['geography_group']).\
        set_index('geography')

    df['% Total Jobs (1-mile walk)'] = df['Jobs within 1-mile Walk'].apply(lambda x: f'{x / tot_jobs * 100:,.1f}' + '%')
    df['% Total Jobs (3-mile bike)'] = df['Jobs within 3-mile Bike'].apply(lambda x: f'{x / tot_jobs * 100:,.1f}' + '%')

    return df

In [11]:
bp_job_access_geog(df_access_bp,'CountyName')

,Jobs within 1-mile Walk,Jobs within 3-mile Bike,% Total Jobs (1-mile walk),% Total Jobs (3-mile bike)
geography,,,,
King,"18,515.8","84,913.2",0.9%,3.9%
Kitsap,"1,236.0","8,014.4",0.1%,0.4%
Outside Region,0.0,31.0,0.0%,0.0%
Pierce,"2,508.0","18,163.2",0.1%,0.8%
Snohomish,"2,005.8","17,702.8",0.1%,0.8%
Region,"11,163.9","54,253.8",0.5%,2.5%


In [12]:
df_rgc = bp_job_access_geog(df_access_bp,'rgc_binary')
df = bp_job_access_geog(df_access_bp,'GrowthCenterName')

pd.concat([df_rgc, df.loc[~df.index.isin(['Region','Not in RGC'])]], axis=0)

,Jobs within 1-mile Walk,Jobs within 3-mile Bike,% Total Jobs (1-mile walk),% Total Jobs (3-mile bike)
geography,,,,
Region,"11,163.9","54,253.8",0.5%,2.5%
Not in RGC,"2,912.2","34,868.4",0.1%,1.6%
In RGC,"88,877.7","236,823.8",4.1%,11.0%
Auburn,"10,541.0","40,324.1",0.5%,1.9%
Bellevue,"59,188.0","110,699.5",2.7%,5.1%
Bothell Canyon Park,"8,538.5","21,748.3",0.4%,1.0%
Bremerton,"11,535.8","34,387.3",0.5%,1.6%
Burien,"4,828.8","13,399.8",0.2%,0.6%
Everett,"15,907.7","39,722.1",0.7%,1.8%


In [13]:
bp_job_access_geog(df_access_bp,'rg_proposed')

,Jobs within 1-mile Walk,Jobs within 3-mile Bike,% Total Jobs (1-mile walk),% Total Jobs (3-mile bike)
geography,,,,
Region,"11,163.9","54,253.8",0.5%,2.5%
CitiesTowns,983.7,"8,947.8",0.0%,0.4%
Core,"3,414.8","28,587.0",0.2%,1.3%
HCT,"1,427.7","15,237.5",0.1%,0.7%
Metro,"29,432.4","128,074.5",1.4%,5.9%
UU,442.6,"6,815.6",0.0%,0.3%


In [14]:
bp_job_access_geog(df_access_equity_bp,'Equity Geography')

,Jobs within 1-mile Walk,Jobs within 3-mile Bike,% Total Jobs (1-mile walk),% Total Jobs (3-mile bike)
geography,,,,


## Intersection Density

In [15]:
buffered_parcels = pl.read_csv(output_path / 'landuse/buffered_parcels.txt', 
                               separator=' ',
                               columns=['parcelid','nodes3_2','nodes4_2','hh_p'])


async_engine = create_engine('sqlite:///' + summary_config['sc_run_path'] + '/inputs/db/' + config['db_name'])

list_cols = ['ParcelID','CountyName','GrowthCenterName','rg_proposed'] + equity_geogs
parcel_geog = pl.read_database(
    query= f"SELECT {', '.join(list_cols)} FROM " + "parcel_" + config["base_year"] + "_geography",
    connection=async_engine.connect()
)

In [16]:
df_intersection = buffered_parcels.join(parcel_geog, left_on='parcelid', right_on='ParcelID').to_pandas()

# Total intersections within 1/2 mile buffer
df_intersection['intersections_wt'] = (df_intersection['nodes3_2'] + df_intersection['nodes4_2']) * df_intersection['hh_p']

In [17]:
def intersection_density(geog):
    df = df_intersection.groupby(geog)[['intersections_wt', 'hh_p']].sum().reset_index()
    df['Intersections'] = df['intersections_wt']/df['hh_p']
    
    return df[[geog] + ['Intersections']]

In [18]:
intersection_density('CountyName')

,CountyName,Intersections
0,King,155.0
1,Kitsap,50.6
2,Outside Region,3.6
3,Pierce,80.1
4,Snohomish,73.4


In [19]:
intersection_density('GrowthCenterName')

,GrowthCenterName,Intersections
0,Auburn,192.4
1,Bellevue,273.3
2,Bothell Canyon Park,68.8
3,Bremerton,168.0
4,Burien,175.8
5,Everett,160.6
6,Federal Way,133.9
7,Greater Downtown Kirkland,166.5
8,Issaquah,NaN
9,Kent,214.0


In [20]:
intersection_density('rg_proposed')

,rg_proposed,Intersections
0,CitiesTowns,64.6
1,Core,103.2
2,HCT,84.2
3,Metro,201.6
4,UU,56.1


In [21]:
df_intersection_equity = df_intersection.loc[df_intersection['disability_geog_vs_reg_total']>=0].copy()

# get total intersections by equity geography
df1 = df_intersection_equity[equity_geogs].apply(lambda x: x * df_intersection_equity['intersections_wt']).sum().reset_index()
df1.columns = ['Equity Group', 'intersections_wt']

# get total households by equity geography
df2 = df_intersection_equity[equity_geogs].apply(lambda x: x * df_intersection_equity['hh_p']).sum().reset_index()
df2.columns = ['Equity Group', 'hh_p']

df_in = df1.merge(df2, on='Equity Group')
df_in['Inside Equity Geography'] = df_in['intersections_wt']/df_in['hh_p']

# outside equity geographies
df1_out = df_intersection_equity[equity_geogs].apply(lambda x: (1-x) * df_intersection_equity['intersections_wt']).sum().reset_index()
df1_out.columns = ['Equity Group', 'intersections_wt']
df2_out = df_intersection_equity[equity_geogs].apply(lambda x: (1-x) * df_intersection_equity['hh_p']).sum().reset_index()
df2_out.columns = ['Equity Group', 'hh_p']
df_out = df1_out.merge(df2_out, on='Equity Group')
df_out['Outside Equity Geography'] = df_out['intersections_wt']/df_out['hh_p']

df = df_in[['Equity Group','Inside Equity Geography']].merge(
    df_out[['Equity Group','Outside Equity Geography']], on='Equity Group')

df

,Equity Group,Inside Equity Geography,Outside Equity Geography
0,youth_geog_vs_reg_total,77.6,151.7
1,elderly_geog_vs_reg_total,100.9,134.2
2,english_geog_vs_reg_total,120.4,117.1
3,racial_geog_vs_reg_total,136.0,103.5
4,racial_geog_vs_50_percent,139.7,108.7
5,poverty_geog_vs_reg_total,127.1,112.5
6,poverty_geog_vs_50_percent,197.9,116.6
7,disability_geog_vs_reg_total,115.4,120.9
